# Monitoring chlorophyll-*a* in Australian waterbodies

**Notebook currently compatible with the `DEA Sandbox` environment (untested on `NCI`)**

### General advice (delete this cell before submitting for review)

- When choosing a location for your analysis, **select an area that has data on both the `NCI` and `DEA Sandbox`** to allow your code to be run on both environments. 
For example, you can check this for Landsat using the [DEA Explorer](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3/1990) (use the drop-down menu to view all products). 
As of September 2019, the `DEA Sandbox` has a single year of continental Landsat data for 2015-16, and the full 1987-onward time-series for three locations (Perth WA, Brisbane QLD, and western NSW).
- When writing in Markdown cells, start each sentence is on a **new line**.
This makes it easy to see changes through git commits.
- Use Australian English in markdown cells and code comments.
- Use the [PEP8 standard](https://www.python.org/dev/peps/pep-0008/) for code. To make sure all code in the notebook is consistent, you can use the `jupyterlab_code_formatter` tool: select each code cell, then click `Edit` and then one of the `Apply X Formatter` options (`YAPF` or `Black` are recommended). This will reformat the code in the cell to a consistent style.
- In the final notebook cell, include a set of relevant tags which are used to build the DEA User Guide's [Tag Index](https://docs.dea.ga.gov.au/genindex.html). 
Use all lower-case, seperate words with spaces, and where possible re-use existing tags.
Ensure the tags cell below is in `Raw` format, rather than `Markdown` or `Code`.


### Background
An *optional* overview of the scientific, economic or environmental management issue or challenge being addressed by Digital Earth Australia. 
For `Beginners_Guide` or `Frequently_Used_Code` notebooks, this may include information about why the particular technique or approach is useful or required. 
If you need to cite a scientific paper or link to a website, use a persistent DOI link if possible and link in-text (e.g. [Dhu et al. 2017](https://doi.org/10.1080/20964471.2017.1402490)).

### The Problem
A _compulsory_ description of the notebook, including a brief overview of how Digital Earth Australia helps to address the problem set out above, and a run-down of the tools/methods being demonstrated below.

### Digital Earth Australia use case


### Technical details
* **Products used:** [`s2a_ard_granule`](https://explorer.sandbox.dea.ga.gov.au/s2a_ard_granule), [`s2b_ard_granule`](https://explorer.sandbox.dea.ga.gov.au/s2b_ard_granule)
* **Analyses used:** NDCI index

## Getting started
**To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 
When you finished the analysis, you can return to the "Set up analysis" cell, modify some values (e.g. choose a different location or time period to analyse) and re-run the analysis.
There are additional instructions on modifying the notebook at the end.**

### Load packages

In [ ]:
%matplotlib inline

import datacube
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import xarray as xr

sys.path.append("../Scripts")
from dea_plotting import display_map, rgb
from dea_datahandling import load_ard

### Connect to the datacube
Give your datacube app a unique name that is consistent with the purpose of the notebook.

In [ ]:
dc = datacube.Datacube(app="Chlorophyll_monitoring")

### Analysis parameters

An *optional* section to inform the user of any parameters they'll need to configure to run the notebook:
* `param_name_1`: Simple description (e.g. `example_value`). Advice about appropriate values to choose for this parameter.
* `param_name_2`: Simple description (e.g. `example_value`). Advice about appropriate values to choose for this parameter.


In [ ]:
# Menindee
latitude = (-32.415, -32.273)
longitude = (142.228, 142.409)

# Lake Baroon
# -26.708465, 152.870373
# latitude = (-26.725, -26.695)
# longitude = (152.855, 152.890)


time = ('2017-01-01', '2018-12-01')

## View the selected location
The next cell will display the selected area on an interactive map.
Feel free to zoom in and out to get a better understanding of the area you'll be analysing.
Clicking on any point of the map will reveal the latitude and longitude coordinates of that point.

In [ ]:
display_map(x=longitude, y=latitude)

## Load and view Sentinel 2 data

In [ ]:
products = ['s2a_ard_granule', 's2b_ard_granule']

query = {
    "x": longitude,
    "y": latitude,
    "time": time,
    "measurements": [
        "nbart_red_edge_1",
        "nbart_red",
        "nbart_green",
        "nbart_blue",
        "nbart_nir_1",
        "nbart_swir_2",
        "fmask"
    ],
    "output_crs": "EPSG:3577",
    "resolution": (-20, 20)
}

ds_s2 = load_ard(dc, products=products, min_gooddata=0.7, **query)

In [ ]:
#ds_s2['time'] = ds_s2.indexes['time'].normalize()
print(ds_s2)

### Plot example timestep in true colour

In [ ]:
rgb(ds_s2, index=47)

## Compute band indices

We'll need to calculate the modified normalised difference water index (MNDWI) and the normalised difference chlorophyll index (NDCI).

In [ ]:
ds_s2['MNDWI'] = (ds_s2.nbart_green - ds_s2.nbart_swir_2)/(ds_s2.nbart_green + ds_s2.nbart_swir_2)
ds_s2['NDCI'] = (ds_s2.nbart_red_edge_1 - ds_s2.nbart_red)/(ds_s2.nbart_red_edge_1 + ds_s2.nbart_red)

In [ ]:
ds_s2.MNDWI.isel(time=0).plot(cmap='RdBu')
plt.show()

## Build summary plot

In [ ]:
# Constants for calculating water body area
pixel_length = query["resolution"][1]  # in metres
m_per_km = 1000  # conversion from metres to kilometres
area_per_pixel = pixel_length**2 / m_per_km**2

In [ ]:
# Compute the total water area 
ds_s2_waterarea = ds_s2.where(ds_s2.MNDWI > 0.0)

waterarea = ds_s2_waterarea.MNDWI.count(dim=["x", "y"]).rolling(time=3, center=True, min_periods=1).median().values * area_per_pixel

average_ndci = ds_s2_waterarea.NDCI.mean(dim=["x", "y"]).values

fig, axes = plt.subplots(1, 1, figsize=(12, 4))
cmap = plt.get_cmap('viridis')
normal = plt.Normalize(vmin=-0.1, vmax=0.3)

dates=matplotlib.dates.date2num(ds_s2_waterarea.time.values)
axes.plot_date(x=dates, y=waterarea, color='black', linestyle='-', marker='')

color_vals = np.linspace(0, 1, 100)

for ii, thresh in enumerate(np.linspace(-0.1, 0.3, 100)):
    im = axes.fill_between(dates, 0, waterarea, where=(average_ndci >= thresh), norm=normal, facecolor=cmap(color_vals[ii]), alpha=1)

cax, _ = matplotlib.colorbar.make_axes(axes)
cb2 = matplotlib.colorbar.ColorbarBase(cax, cmap=cmap, norm=normal)

## Compare times

In [ ]:
def closest_date(list_of_dates, desired_date):
    return min(list_of_dates, key=lambda x: abs(x - desired_date))

def date_index(list_of_dates, known_date):
    return(np.where(list_of_dates == known_date)[0][0])

time_1 = closest_date(ds_s2.time.values, np.datetime64('2017-01-15'))

time_2 = closest_date(ds_s2.time.values, np.datetime64('2018-01-15'))

time_xr = xr.DataArray([time_1, time_2], dims=['time'])

ds_s2_waterarea.NDCI.sel(time=time_xr).plot.imshow('x', 'y', col='time', cmap=cmap, vmin=-0.1, vmax=0.3, col_wrap=2, robust=True, figsize=(12, 5))

plt.show()

In [ ]:
time_1_idx = date_index(ds_s2.time.values, time_1)
time_2_idx = date_index(ds_s2.time.values, time_2)

rgb(ds_s2, index=time_1_idx)
rgb(ds_s2, index=time_2_idx)

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** September 2019

**Compatible `datacube` version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)